# 02 Correct Network Errors with Project Cards

In [1]:
import os
import sys
import yaml
import pickle
import glob

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O and Parameters

In [4]:
root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_02')
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-10-27 14:03:05, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 14:03:05, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


## Read Version 00 Scenario

In [6]:
version_00_pickle_file_name = os.path.join(input_dir, 'working_scenario_00.pickle')
v_00_scenario = pickle.load(open(version_00_pickle_file_name, 'rb'))

In [7]:
v_00_scenario.road_net.links_df.shape

(1634769, 30)

In [8]:
v_01_scenario = v_00_scenario

## Create Scenario and Apply Project Cards

In [ ]:
# TODO: replace with tag fetch once debug is done
name_list = glob.glob(card_dir + '/*.yml')
name_list.remove('/Users/wsp/Documents/GitHub/travel-model-two-networks/project_cards/year_2015_i80_wb_hov_ala.yml')
name_list.remove('/Users/wsp/Documents/GitHub/travel-model-two-networks/project_cards/year_2015_bus_only_ggt_on_us101.yml')
name_list.remove('/Users/wsp/Documents/GitHub/travel-model-two-networks/project_cards/year_2015_attributes_exclude_trucks_sr85.yml')
name_list.remove('/Users/wsp/Documents/GitHub/travel-model-two-networks/project_cards/year_2015_managed_lane_i80_ramp_to_bay_bridge.yml')
name_list.remove('/Users/wsp/Documents/GitHub/travel-model-two-networks/project_cards/year_2015_attributes_exclude_trucks_i580.yml')
name_list.remove('/Users/wsp/Documents/GitHub/travel-model-two-networks/project_cards/year_2015_bus_only_transbay_terminal_ramps.yml')
card_list = [] 
for name in name_list:
    card = ProjectCard.read(os.path.join(card_dir, name), validate = False)
    card_list.append(card)

In [12]:
for card in card_list:
    v_01_scenario.apply_project(card)

2020-10-27 14:05:26, INFO: Applying Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-10-27 14:05:26, INFO: Applying Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-10-27 14:05:26, INFO: Applying Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-10-27 14:05:26, INFO: Applying Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-10-27 14:05:26, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-10-27 14:05:26, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [13]:
v_01_scenario.applied_projects

['Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma']

In [14]:
v_00_scenario.road_net.links_df.shape

(1634769, 36)

In [15]:
v_00_scenario.road_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'shape_id', 'geometry', 'ft', 'useclass',
       'assignable', 'transit', 'managed', 'ML_lanes', 'segment_id',
       'ML_useclass', 'ML_access', 'ML_egress'],
      dtype='object')

## Make Travel Model Networks

In [16]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_01_scenario.road_net, 
    parameters = parameters)

2020-10-27 14:11:01, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 14:11:01, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


Sijia - TO DO test append centroid and centroid connectors before vs after project cards are applied, we don't want to renumber centroid and centroid connectors every time when new links and nodes are added through project cards

In [17]:
model_net = mtc.add_centroid_and_centroid_connector(
    roadway_network = model_net,
    parameters = parameters)

2020-10-27 14:12:15, INFO: Adding centroid and centroid connector to standard network
2020-10-27 14:12:15, INFO: Adding centroid and centroid connector to standard network
2020-10-27 14:12:15, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 14:12:15, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 14:12:29, INFO: Finished adding centroid and centroid connectors
2020-10-27 14:12:29, INFO: Finished adding centroid and centroid connectors


net = mtc.calculate_facility_type(
    roadway_network = model_net,
    parameters = parameters,
    network_variable = 'ft')

net.links_df.ft.value_counts()

net = mtc.calculate_cntype(
    roadway_network = model_net,
    parameters = parameters,
    overwrite=True)

v_01_scenario.road_net.links_df.cntype.value_counts()

In [21]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2020-10-27 14:14:11, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2020-10-27 14:14:11, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2020-10-27 14:14:11, INFO: Creating managed lane network.
2020-10-27 14:14:11, INFO: Creating managed lane network.
2020-10-27 14:14:11, INFO: Creating network with duplicated managed lanes
2020-10-27 14:14:11, INFO: Creating network with duplicated managed lanes


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\wang

2020-10-27 14:15:13, INFO: Calculating Facility Type
2020-10-27 14:15:13, INFO: Calculating Facility Type
2020-10-27 14:15:13, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 14:15:13, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 14:15:13, INFO: Variable 'ft' updated for some links. Returning without overwriting for those links. Calculating for other links
2020-10-27 14:15:13, INFO: Variable 'ft' updated for some links. Returning without overwriting for those links. Calculating for other links
2020-10-27 14:17:28, INFO: Finished calculating roadway class variable: ft
2020-10-27 14:17:28, INFO: Finished calculating roadway class variable: ft
2020-10-27 14:17:29, INFO: Determining assignable
2020-10-27 14:17:29, INFO: Determining assignable
2020-10-27 14:17:29, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 14:17:29, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/githu

In [22]:
model_net.links_df.shape

(1634898, 53)

## Write to Disk

## As shapefile

In [ ]:
v_01_scenario.road_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry"],
    node_output_variables = ["model_node_id", "geometry"],
    data_to_csv = False,
    data_to_dbf = True,
)

## As fixed width (for CUBE)

In [ ]:
v_01_scenario.road_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

In [ ]:
from subprocess import Popen

os.chdir(output_dir)
p = Popen("make_cube.bat")
p.communicate()

## As pickle

In [ ]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_01.pickle')
pickle.dump(v_01_scenario, open(working_scenario_filename, 'wb'))